In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, PolynomialFeatures
from sklearn.ensemble import StackingClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import Pipeline

In [15]:
# Загрузка данных
df = pd.read_csv("data/train.csv")


df = df.drop(columns=["PassengerId", "Name", "Ticket", "Cabin"])
df["Age"] = df["Age"].fillna(df["Age"].median())
df["Embarked"] = df["Embarked"].fillna(df["Embarked"].mode()[0])

# Преобразование категориальных признаков с помощью LabelEncoder
label_encoders = {}
for col in ["Sex", "Embarked"]:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le


In [16]:
X = df.drop(columns=["Survived"])
y = df["Survived"]

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

Stacking Classifier (Стекинг)

В вашем примере используются два базовых классификатора (случайный лес и градиентный бустинг), а их прогнозы передаются в логистическую регрессию, которая делает финальное предсказание.

In [18]:
base_learners = [
    ("rf", RandomForestClassifier(n_estimators=100, random_state=42)),
    ("gb", GradientBoostingClassifier(n_estimators=100, random_state=42))
]
stacking_model = StackingClassifier(estimators=base_learners, final_estimator=LogisticRegression())
stacking_model.fit(X_train, y_train)
y_pred_stacking = stacking_model.predict(X_test)
stacking_accuracy = accuracy_score(y_test, y_pred_stacking)
print(f"Stacking Accuracy: {stacking_accuracy:.4f}")

Stacking Accuracy: 0.8101


/Users/belladavidyan/Documents/TMO_Labs/myenv/lib/python3.9/site-packages/sklearn/linear_model/_linear_loss.py:200: RuntimeWarning: divide by zero encountered in matmul
  raw_prediction = X @ weights + intercept
/Users/belladavidyan/Documents/TMO_Labs/myenv/lib/python3.9/site-packages/sklearn/linear_model/_linear_loss.py:200: RuntimeWarning: overflow encountered in matmul
  raw_prediction = X @ weights + intercept
/Users/belladavidyan/Documents/TMO_Labs/myenv/lib/python3.9/site-packages/sklearn/linear_model/_linear_loss.py:200: RuntimeWarning: invalid value encountered in matmul
  raw_prediction = X @ weights + intercept


 MLPClassifier (Многослойный персептрон)
 
Каждое предсказание проходит через несколько слоев нейронов, каждый из которых обучается на предыдущих ошибках.

In [19]:
mlp_model = MLPClassifier(hidden_layer_sizes=(50, 30), max_iter=500, random_state=42)
mlp_model.fit(X_train, y_train)
y_pred_mlp = mlp_model.predict(X_test)
mlp_accuracy = accuracy_score(y_test, y_pred_mlp)
print(f"MLP Accuracy: {mlp_accuracy:.4f}")

MLP Accuracy: 0.7877


/Users/belladavidyan/Documents/TMO_Labs/myenv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/belladavidyan/Documents/TMO_Labs/myenv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/belladavidyan/Documents/TMO_Labs/myenv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/Users/belladavidyan/Documents/TMO_Labs/myenv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/belladavidyan/Documents/TMO_Labs/myenv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/belladavidyan/Documents/TMO_Labs/myenv/lib/python3.9/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  

In [20]:
X_train_np = X_train.to_numpy(dtype=float)  # Преобразуем в numpy массив
X_test_np = X_test.to_numpy(dtype=float)
y_train_np = y_train.to_numpy(dtype=float).ravel()  # Делаем одномерным
y_test_np = y_test.to_numpy(dtype=float).ravel()

. Combi (линейный метод GMDH)
Описание: Combi — это линейный метод из семейства GMDH (генеративных моделей данных), который строит модель, используя линейные комбинации признаков.

Как работает: Метод создает линейные модели для решения задачи классификации или регрессии, минимизируя ошибку на каждом шаге.

In [21]:
combi_pipeline = Pipeline([
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('feature_selection', SelectKBest(f_classif, k=5)),  # Выбираем 5 лучших признаков
    ('model', RidgeClassifier(random_state=42))
])

combi_pipeline.fit(X_train, y_train)
y_pred_combi = combi_pipeline.predict(X_test)
combi_accuracy = accuracy_score(y_test, y_pred_combi)
print(f"COMBI-like Accuracy: {combi_accuracy:.4f}")

COMBI-like Accuracy: 0.7765


Mia (нелинейный метод GMDH)

Описание: Mia — это нелинейная модель GMDH, использующая более сложные структуры для построения зависимости между признаками.

Как работает: В отличие от линейных методов, Mia создает более гибкие модели, которые могут учитывать нелинейные зависимости в данных.

In [22]:
mia_model = GradientBoostingClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=3,
    random_state=42
)

mia_model.fit(X_train, y_train)
y_pred_mia = mia_model.predict(X_test)
mia_accuracy = accuracy_score(y_test, y_pred_mia)
print(f"MIA-like Accuracy: {mia_accuracy:.4f}")

MIA-like Accuracy: 0.7989
